# Simulation Experiments

## Load Model

In [ ]:
import pysd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import numpy as np

current_dir = Path.cwd()
BASE_DIR = current_dir.parent if current_dir.name == 'notebooks' else current_dir
os.chdir(BASE_DIR)

# Load the default parameters
with open(BASE_DIR / 'params.yaml', 'r') as file:
    BASE_PARAMS = yaml.safe_load(file)

model = pysd.load('src/model.py')



## Highest Tax & Emission Trade-off

- Binary search over a tax range to find the maximum sustainable tax. 
- Simulate all taxes up to frontier and compute CO2 reduction vs profitability.

### Max Tax

In [ ]:
from src.utils import calculate_max_viable_tax

tax_frontier = calculate_max_viable_tax(model, BASE_PARAMS)

print(tax_frontier)

### CO2 Reduction vs Profitability

In [ ]:
return_columns = [
    "cumulative_co2",
    "cumulative_profit",
    "viability_flag"
]

baseline_run = model.run(params=BASE_PARAMS, return_columns=return_columns)

co2_base = baseline_run["cumulative_co2"].iloc[-1]
profit_base = baseline_run["cumulative_profit"].iloc[-1]

# Vector of taxes to simulate. Using 30 points for now.
tax_grid = np.linspace(BASE_PARAMS["carbon_tax_rate"], tax_frontier, 10)

results_list = []

for tax in tax_grid:
    params = BASE_PARAMS.copy()
    params["carbon_tax_rate"] = tax
    sim_results = model.run(
        params=params,
        return_columns=return_columns
    )


    co2 = sim_results["cumulative_co2"].iloc[-1]
    profit = sim_results["cumulative_profit"].iloc[-1]

    viability_flag = sim_results["viability_flag"].iloc[-1]

    co2_reduction_pct = 100 * (1 - co2 / co2_base)
    profit_change_pct = 100 * ((profit - profit_base) / profit_base)

    results_list.append({
        "tax": tax,
        "co2": co2,
        "profit": profit,
        "co2_reduction_pct": co2_reduction_pct,
        "profit_change_pct": profit_change_pct,
        "viable": viability_flag
    })

results_df = pd.DataFrame(results_list)
# assert (results_df["viable"] == 1).all(), "Some simulations are not viable"



### Visualisation

In [ ]:
fig, ax = plt.subplots()

sc = ax.scatter(
    results_df["co2_reduction_pct"],
    results_df["profit_change_pct"],
    c=results_df["tax"],  # color by tax level
)

plt.title("CO₂ Reduction vs Profitability Trade-off Curve")

# Fit a quadratic curve to the trade-off
x = results_df["co2_reduction_pct"].values
y = results_df["profit_change_pct"].values

# Sort for smooth curve plotting
sort_idx = np.argsort(x)
x_sorted = x[sort_idx]
y_sorted = y[sort_idx]

# Fit quadratic polynomial
coef_quad = np.polyfit(x_sorted, y_sorted, 2)
y_fitted = np.polyval(coef_quad, x_sorted)

# Plot the fitted trade-off curve
ax.plot(x_sorted, y_fitted, 'k--', linewidth=2, label='Quadratic fit', alpha=0.8)

ax.axhline(0, linestyle="--", color='gray', alpha=0.5)  # zero profit-change line
ax.set_xlabel("CO₂ reduction vs baseline (%)")
ax.set_ylabel("Profit change vs baseline (%)")
ax.legend()
plt.colorbar(sc, label="Carbon tax (¥/tCO₂)")

### Interpretation

Plot shows diminishing marginal profit returns of increasing tax. Shows most efficient at around 2000-3000 yen.

In [ ]:
x = results_df["co2_reduction_pct"]
y = results_df["profit_change_pct"]

coef_linear = np.polyfit(x, y, 1)
y_linear = np.polyval(coef_linear, x)

ss_res = np.sum((y - y_linear) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r_squared = 1 - ss_res / ss_tot

print(f"R-squared: {r_squared}")

coef_quad = np.polyfit(x, y, 2)
y_quad = np.polyval(coef_quad, x)

ss_res_q = np.sum((y - y_quad)**2)
r2_quad = 1 - ss_res_q/ss_tot

print("Quadratic fit R^2:", r2_quad)


# Sort by CO2 reduction
df_sorted = results_df.sort_values("co2_reduction_pct")
x = df_sorted["co2_reduction_pct"].values
y = df_sorted["profit_change_pct"].values

# second finite difference (approx curvature sign)
second_diff = np.diff(y, 2)  # length N-2
print("Mean second difference:", second_diff.mean())
print("Max abs second difference:", np.abs(second_diff).max())

## Sensitivity & Leverage Points

In [ ]:
from src.utils import one_at_a_time_sensitivity_analysis

subsystem_map = {
    "carbon_content_of_fuel": "Fuel CI",
    "baseline_fuel_efficiency": "Fuel Efficiency (Baseline)",
    "max_efficiency": "Fuel Efficiency (Potential)",
    "nonfuel_cost_per_km": "Fixed Costs",
    "pretax_fuel_price": "Fuel Market",
    "desired_passthrough_share": "Pass-Through",
}

# Tax levels: we choose levels within the range of the tax frontier
tax_levels = np.linspace(BASE_PARAMS["carbon_tax_rate"], tax_frontier, 5)

key_params_policy = {
    # Fuel supply / CI
    "carbon_content_of_fuel": np.linspace(BASE_PARAMS["carbon_content_of_fuel"] * 0.7,
                               BASE_PARAMS["carbon_content_of_fuel"] * 1.3, 5),

    # Fleet efficiency: starting point + potential
    "baseline_fuel_efficiency": np.linspace(BASE_PARAMS["baseline_fuel_efficiency"] * 0.8,
                                            BASE_PARAMS["baseline_fuel_efficiency"] * 1.2, 5),
    "max_efficiency": np.linspace(BASE_PARAMS["max_efficiency"] * 0.9,
                                  BASE_PARAMS["max_efficiency"] * 1.1, 5),

    # Cost structure
    "nonfuel_cost_per_km": np.linspace(BASE_PARAMS["nonfuel_cost_per_km"] * 0.8,
                                       BASE_PARAMS["nonfuel_cost_per_km"] * 1.2, 5),

    # Fuel market
    "pretax_fuel_price": np.linspace(BASE_PARAMS["pretax_fuel_price"] * 0.8,
                                     BASE_PARAMS["pretax_fuel_price"] * 1.2, 5),

    # Market / pass-through
    "desired_passthrough_share": np.linspace(BASE_PARAMS["desired_passthrough_share"] * 0.6,
                                             BASE_PARAMS["desired_passthrough_share"] * 1.4, 5),
}

sensitivity_params = BASE_PARAMS.copy()

results = one_at_a_time_sensitivity_analysis(model, sensitivity_params, key_params_policy, tax_levels)

# Calculate normalized influence (range of profit_change_pct) for each parameter at each tax level
influence_data = []
for tax in tax_levels:
    tax_data = results[results['tax'] == tax]
    for param_name in key_params_policy.keys():
        param_data = tax_data[tax_data['param_name'] == param_name]['profit_change_pct']
        if len(param_data) > 0:
            influence = abs(param_data.max() - param_data.min())  # Range as influence measure
            influence_data.append({
                'tax': tax,
                'param_name': subsystem_map.get(param_name, param_name),
                'influence': influence
            })

# Create DataFrame and pivot for plotting
influence_df = pd.DataFrame(influence_data)
plot_df = influence_df.pivot(index='tax', columns='param_name', values='influence')

# Normalize by total influence at each tax level (so bars sum to 1)
plot_df_normalized = plot_df.div(plot_df.sum(axis=1), axis=0)

# Create the plot
plt.figure(figsize=(10, 6))
plot_df_normalized.plot(
    kind="bar",
    stacked=True,
    colormap="tab20",
    ax=plt.gca()
)

plt.ylabel("Normalized Influence on Profit")
plt.xlabel("Carbon Tax Level (¥/tCO₂)")
plt.title("Subsystem Influence Profile Across Carbon Tax Levels")
plt.legend(title="Subsystem", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig(f"{BASE_DIR}/figures/results/profit_subsystem_sensitivity.png")
plt.show()



In [ ]:
# Calculate normalized influence (range of co2_reduction_pct) for each parameter at each tax level
influence_data_co2 = []
for tax in tax_levels:
    tax_data = results[results['tax'] == tax]
    for param_name in key_params_policy.keys():
        param_data = tax_data[tax_data['param_name'] == param_name]['co2_reduction_pct']
        if len(param_data) > 0:
            influence = abs(param_data.max() - param_data.min())  # Range as influence measure
            influence_data_co2.append({
                'tax': tax,
                'param_name': subsystem_map.get(param_name, param_name),
                'influence': influence
            })

# Create DataFrame and pivot for plotting
influence_df_co2 = pd.DataFrame(influence_data_co2)
plot_df_co2 = influence_df_co2.pivot(index='tax', columns='param_name', values='influence')

# Normalize by total influence at each tax level (so bars sum to 1)
plot_df_co2_normalized = plot_df_co2.div(plot_df_co2.sum(axis=1), axis=0)

# Create the plot
plt.figure(figsize=(10, 6))
plot_df_co2_normalized.plot(
    kind="bar",
    stacked=True,
    colormap="tab20",
    ax=plt.gca()
)

plt.ylabel("Normalized Influence on CO₂ Reduction")
plt.xlabel("Carbon Tax Level (¥/tCO₂)")
plt.title("Subsystem Influence Profile on CO₂ Reduction Across Carbon Tax Levels")
plt.legend(title="Subsystem", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


## Tax Adjustment Rules

We explore four tax adjustment rules and examine how they compare to static tax levels in terms of outcomes.

For each adjustment rule we:
1. Simulate 10 years with a time-varying carbon tax, $\tau(t)$.
2. Compute time-averaged tax over 10 years (mean of monthly taxes), cumulative CO₂ at t=120, cumulative profit at t=120, and viability.
3. Compare that to a static tax equal to the same average level:
 
   $$\bar{\tau} = \frac{1}{T} \sum_{t=1}^T{\tau(t)}$$ 
 
4. Run the model once with `carbon_tax_rate` $=\bar{\tau}$ as a constant and determine if the adaptive tax achieves higher cumulative profit for similar emissions, lower emissions for similar profit or both (best case, this means it's Pareto-better).

The adaptive tax rules that will be examined is:
1. Simple annual step increase:

   $$\tau_{t+12} = \min(\tau_t + \delta_{\text{step}}, \tau_{\max})$$

   Tax climbs by a fixed amount each year until it reaches a cap mimicing a pre-announced tax ramp.

2. Percentage growth rule: 

   $$\tau_{t+12} = \tau_t(1 + g)$$

   Tax grows at g% per year mimicing indexing tax to some growth rule.

3. Profit / margin-based rule:

   $$
   \tau_{t+12} = \begin{cases}
   \tau_t + \Delta_{\uparrow} & \text{if } \text{rolling\_margin}_t > m^* + \text{band} \\
   \tau_t & \text{if } m^* - \text{band} \leq \text{rolling\_margin}_t \leq m^* + \text{band} \\
   \tau_t - \Delta_{\downarrow} & \text{if } \text{rolling\_margin}_t < m^* - \text{band}
   \end{cases}
   $$

   This rule raises tax when there is 'room' (high margin), freezes or eases if operators are under stress (low margin).

4. Emission-path–based rule:

   $$
   \tau_{t+12} = \begin{cases}
   \tau_t + \Delta_{\uparrow} & \text{if } E(t) > E^{\text{target}}(t) + \text{band} \\
   \tau_t & \text{if } |E(t) - E^{\text{target}}(t)| \leq \text{band} \\
   \tau_t - \Delta_{\downarrow} & \text{if } E(t) < E^{\text{target}}(t) - \text{band}
   \end{cases}
   $$

   This is like a simple feedback controller on emissions, where $E(t)$ is the cumulative CO₂ emissions at time $t$ and $E^{\text{target}}(t)$ is the target cumulative emissions path.
 

### Simple annual step increase

In [ ]:
from pysd.py_backend.output import ModelOutput

output = ModelOutput()

model.set_stepper(
    output,
    step_vars=["carbon_tax_rate"],
    final_time=120,
)
params = BASE_PARAMS.copy()

tax_rate_current = BASE_PARAMS["carbon_tax_rate"]
tax_rate_current_list = []

for t in range(120):
    model.step(1, {"carbon_tax_rate": tax_rate_current})
    tax_rate_current = np.minimum(tax_rate_current + 2, tax_frontier)
    tax_rate_current_list.append(tax_rate_current)

results_step_increase = output.collect(model)
co2_step_increase = results_step_increase["Cumulative CO2"].iloc[-1]
profit_step_increase = results_step_increase["Cumulative Profit"].iloc[-1]
viability_step_increase = results_step_increase["Viability Flag"].iloc[-1]

# Calculate time-averaged tax, we use this as the static tax level to compare against
time_averaged_tax = np.mean(tax_rate_current_list)

# Run the model with the static tax level
params = BASE_PARAMS.copy()
params["carbon_tax_rate"] = time_averaged_tax
static_run = model.run(params=params, return_columns=return_columns)

co2_static = static_run["cumulative_co2"].iloc[-1]
profit_static = static_run["cumulative_profit"].iloc[-1]
viability_static = static_run["viability_flag"].iloc[-1]

comparison = pd.DataFrame({
    "Metric": ["Cumulative CO2", "Cumulative Profit", "Viability", "Time-Averaged Tax"],
    "Step Increase": [
        co2_step_increase,
        profit_step_increase,
        viability_step_increase,
        time_averaged_tax
    ],
    "Static": [
        co2_static,
        profit_static,
        viability_static,
        time_averaged_tax
    ]
})

# Calculate differences
comparison["Absolute Difference"] = comparison["Step Increase"] - comparison["Static"]
comparison["Relative Difference (%)"] = 100 * (comparison["Absolute Difference"] / comparison["Static"])

print(comparison)


In [ ]:
# Compare full trajectories
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# CO2 trajectories
axes[0, 0].plot(results_step_increase.index, results_step_increase["Cumulative CO2"], 
                label="Step Increase", linewidth=2)
axes[0, 0].plot(static_run.index, static_run["cumulative_co2"], 
                label="Static", linewidth=2, linestyle="--")
axes[0, 0].set_xlabel("Time (months)")
axes[0, 0].set_ylabel("Cumulative CO2")
axes[0, 0].set_title("Cumulative CO2 Comparison")
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Profit trajectories
axes[0, 1].plot(results_step_increase.index, results_step_increase["Cumulative Profit"], 
                label="Step Increase", linewidth=2)
axes[0, 1].plot(static_run.index, static_run["cumulative_profit"], 
                label="Static", linewidth=2, linestyle="--")
axes[0, 1].set_xlabel("Time (months)")
axes[0, 1].set_ylabel("Cumulative Profit")
axes[0, 1].set_title("Cumulative Profit Comparison")
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Tax rate over time
axes[1, 0].plot(range(len(tax_rate_current_list)), tax_rate_current_list, 
                label="Step Increase", linewidth=2)
axes[1, 0].axhline(time_averaged_tax, label="Static", linewidth=2, linestyle="--", color='orange')
axes[1, 0].set_xlabel("Time (months)")
axes[1, 0].set_ylabel("Carbon Tax Rate")
axes[1, 0].set_title("Tax Rate Comparison")
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Difference over time
co2_diff = results_step_increase["Cumulative CO2"].values - static_run["cumulative_co2"].values
axes[1, 1].plot(static_run.index, co2_diff, linewidth=2, color='red')
axes[1, 1].axhline(0, linestyle="--", color='gray', alpha=0.5)
axes[1, 1].set_xlabel("Time (months)")
axes[1, 1].set_ylabel("CO2 Difference (Step - Static)")
axes[1, 1].set_title("CO2 Difference Over Time")
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Check if step increase is Pareto-better (better on both dimensions)
co2_better = co2_step_increase < co2_static  # Lower is better
profit_better = profit_step_increase > profit_static  # Higher is better

if co2_better and profit_better:
    print("Step increase is Pareto-better: lower CO2 AND higher profit")
elif co2_better:
    print("Step increase: lower CO2 but lower profit")
elif profit_better:
    print("Step increase: higher profit but higher CO2")
else:
    print("Static is Pareto-better: lower CO2 AND higher profit")

We find that step increase is Pareto-better with small relative increases and static is Pareto-better with large relative increases.